# Getting Started with the Icestick

Installation instructions for running Magma on the Lattice IceStick using the yosys and icestorm tools.

Adapted from
[https://github.com/rdaly525/CS448H/blob/master/icestick/README.md](https://github.com/rdaly525/CS448H/blob/master/icestick/README.md)
by Ross Daly

### Lattice Icestick

We will be using the Lattice Icestick ICE40 evaluation board.

![icestick](images/icestick.jpg)

[http://www.latticesemi.com/icestick](http://www.latticesemi.com/icestick)

> USB thumb drive form factor evaluation board - [...] an easy to use, small size board that allows rapid prototyping of system functions at a very low cost using Lattice Semiconductor's iCE40 FPGA family.

### Project Icestorm

A major advantage of using this board is that all the tools needed to program the FPGA are available as open source software that runs on Mac, Linus, and Windoes.

[http://www.clifford.at/icestorm/](http://www.clifford.at/icestorm/)

> Project IceStorm reversed engineered and documented the bitstream
> format of Lattice iCE40 FPGAs. They also wrote a suite of command line tools 
> for manipulating and creating bitstream files. 

> The IceStorm flow includes two other tools: yosys and Arachne-pnr.
> Yosys is an open source verilog synthesis engine that maps verilog to  
> ICE40 primitives.
> Arachne-pnr is a place and router for the ICE40.

## Setup
Installation instructions adapted from [the icestorm website]([http://www.clifford.at/icestorm/](http://www.clifford.at/icestorm/)

#### IceStorm Tools (icepack, icebox, iceprog, icetime, chip databases)

```
$ git clone https://github.com/cliffordwolf/icestorm.git icestorm
$ cd icestorm
$ make -j$(nproc)
$ sudo make install
$ cd ..
```
#### Arachne-PNR (Place & Route)

```
$ git clone https://github.com/cseed/arachne-pnr.git arachne-pnr
$ cd arachne-pnr
$ make -j$(nproc)
$ sudo make install
$ cd ..
```

#### Yosys (Verilog Synthesis)

```
$ git clone https://github.com/cliffordwolf/yosys.git yosys
$ cd yosys
$ make -j$(nproc)
$ sudo make install
$ cd ..
```

Need to add instructions for magma, mantle, and loam.

To test whether the installation is successful, we will compile a Magma program that blinks an LED on the IceStick. 

In [1]:
with open("blink.py", "r") as blink:
    print(blink.read())

from magma import wire
from mantle import Counter
from loam.boards.icestick import IceStick

icestick = IceStick()
icestick.Clock.on()
icestick.D5.on()

main = icestick.main()

N = 22

counter = Counter(N)
wire(counter.O[N-1], main.D5)




Compile the program to produce a verilog file and a PCF (physical constraints file).

In [2]:
!magma -b icestick blink.py

import mantle lattice ice40
import mantle lattice mantle40
compiling FullAdder
compiling Add22Cout
compiling Register22
compiling Counter22
compiling main


In [3]:
with open("build/blink.v", "r") as blink_verilog:
    print(blink_verilog.read())

module FullAdder (input  I0, input  I1, input  CIN, output  O, output  COUT);
wire  inst0_O;
wire  inst1_CO;
SB_LUT4 #(.LUT_INIT(16'h9696)) inst0 (.I0(I0), .I1(I1), .I2(CIN), .I3(1'b0), .O(inst0_O));
SB_CARRY inst1 (.I0(I0), .I1(I1), .CI(CIN), .CO(inst1_CO));
assign O = inst0_O;
assign COUT = inst1_CO;
endmodule

module Add22Cout (input [21:0] I0, input [21:0] I1, output [21:0] O, output  COUT);
wire  inst0_O;
wire  inst0_COUT;
wire  inst1_O;
wire  inst1_COUT;
wire  inst2_O;
wire  inst2_COUT;
wire  inst3_O;
wire  inst3_COUT;
wire  inst4_O;
wire  inst4_COUT;
wire  inst5_O;
wire  inst5_COUT;
wire  inst6_O;
wire  inst6_COUT;
wire  inst7_O;
wire  inst7_COUT;
wire  inst8_O;
wire  inst8_COUT;
wire  inst9_O;
wire  inst9_COUT;
wire  inst10_O;
wire  inst10_COUT;
wire  inst11_O;
wire  inst11_COUT;
wire  inst12_O;
wire  inst12_COUT;
wire  inst13_O;
wire  inst13_COUT;
wire  inst14_O;
wire  inst14_COUT;
wire  inst15_O;
wire  inst15_COUT;
wire  inst16_O;
wire  inst16_COUT;
wire  inst17_O;
wire  inst

In [4]:
with open("build/blink.pcf", "r") as blink_pcf:
    print(blink_pcf.read())

set_io D5 95
set_io CLKIN 21



Next run the yosys tool chain to generate a bitstream.

In [5]:
%%bash
yosys -p 'synth_ice40 -top main -blif build/blink.blif' build/blink.v
arachne-pnr -d 1k -o build/blink.txt -p build/blink.pcf build/blink.blif 
icepack build/blink.txt build/blink.bin


 /----------------------------------------------------------------------------\
 |                                                                            |
 |  yosys -- Yosys Open SYnthesis Suite                                       |
 |                                                                            |
 |  Copyright (C) 2012 - 2018  Clifford Wolf <clifford@clifford.at>           |
 |                                                                            |
 |  Permission to use, copy, modify, and/or distribute this software for any  |
 |  purpose with or without fee is hereby granted, provided that the above    |
 |  copyright notice and this permission notice appear in all copies.         |
 |                                                                            |
 |  THE SOFTWARE IS PROVIDED "AS IS" AND THE AUTHOR DISCLAIMS ALL WARRANTIES  |
 |  WITH REGARD TO THIS SOFTWARE INCLUDING ALL IMPLIED WARRANTIES OF          |
 |  MERCHANTABILITY AND FITNESS. IN NO 

seed: 1
device: 1k
read_chipdb +/share/arachne-pnr/chipdb-1k.bin...
  supported packages: cb121, cb132, cb81, cm121, cm36, cm49, cm81, qn84, swg16tr, tq144, vq100
read_blif build/blink.blif...
prune...
read_pcf build/blink.pcf...
instantiate_io...
pack...

After packing:
IOs          2 / 96
GBs          0 / 8
  GB_IOs     0 / 8
LCs          44 / 1280
  DFF        22
  CARRY      21
  CARRY, DFF 0
  DFF PASS   0
  CARRY PASS 21
BRAMs        0 / 16
WARMBOOTs    0 / 1
PLLs         0 / 1

place_constraints...
promote_globals...
  promoted CLKIN$2, 22 / 22
  promoted 1 nets
    1 clk
  1 globals
    1 clk
realize_constants...
place...
  initial wire length = 861
  at iteration #50: temp = 13.1902, wire length = 208
  at iteration #100: temp = 3.42799, wire length = 159
  at iteration #150: temp = 1.22889, wire length = 75
  at iteration #200: temp = 0.00193609, wire length = 51
  final wire length = 51

After placement:
PIOs       2 / 96
PLBs       12 / 160
BRAMs      0 / 16

  place time 0

Finally, upload the bitstream to the IceStick board.

In [6]:
%%bash
iceprog build/blink.bin

init..
cdone: high
reset..
cdone: low
flash ID: 0x20 0xBA 0x16 0x10 0x00 0x00 0x23 0x51 0x73 0x10 0x23 0x00 0x15 0x00 0x26 0x06 0x06 0x15 0x9F 0x4E
file size: 32220
erase 64kB sector at 0x000000..
programming..
reading..
VERIFY OK
cdone: high
Bye.


If you see the LED blinky, everything has been installed properly.

### ATTENTION MAC USERS
Please read these [notes](http://www.clifford.at/icestorm/notes_osx.html)

One annoying problem on the Mac is the FTDI drivers.

There are three different FTDI drivers for OSX:
1. Apple
2. FTDI
3. open-source

In order to use the icestorm programmer (`iceprog`), you will need to unload the Apple and FTDI drivers.

```
$ sudo kextunload -b com.apple.driver.AppleUSBFTDI
```

To reload the driver later, substitute `kextunload` with `kextload` in the above command reload.

**NB:** You will need to do this again after restarting.